Before running this notebook, ensure that you have ran `population_scrape.py` under the `scripts` directory.

Also, ensure that you have downloaded the income dataset from this [link](https://digital.atlas.gov.au/datasets/digitalatlas::abs-income-including-government-allowances-by-2021-sa2-nov-2023/about) in csv format and put it in the data/landing folder, naming it `income.csv`.

Alternatively, you can download the income dataset directly from this [link](https://drive.google.com/file/d/1EsVPIq_NF0rvhAM29zp2ytJtxiufnqNN/view?usp=sharing). Note that only UniMelb accounts can this second link

### Import Libraries & Read in Files

In [1]:
import warnings
import pandas as pd
import geopandas as gpd
import re

# Suppress all warnings
warnings.filterwarnings('ignore')

income = pd.read_csv('../data/landing/income.csv')
population = pd.read_csv('../data/landing/population.csv')

### Feature Selection & Column Renaming for Income Dataset

In [2]:
# Define regex patterns to extract only the columns that we want
gov_pensions_regex = r'^Government pensions and allowances: (Age pension|Commonwealth rent assistance)'

total_income_regex = (
    r'^Personal income: (Total income earners|Median total income|Mean total income|Total income)'
    r' \(excl\. Government pensions and allowances\)(?!.*(Income share|Quartile|p))'
)

# Find the relevant columns using regex patterns
gov_pensions_columns = income.filter(regex=gov_pensions_regex).columns
total_income_columns = income.filter(regex=total_income_regex).columns

sa2_name  = ['Statistical Areas Level 2 2021 name']

# Combine all selected columns
selected_columns =  sa2_name + list(gov_pensions_columns) + list(total_income_columns)

# Filter to include only the selected features
filtered_income = income[selected_columns]

# Renaming columns
new_income_names = {
    'Statistical Areas Level 2 2021 name': 'sa2_name',
    'Government pensions and allowances: Age pension (no.) (Data year: 2023)': 'gov_age_pension_count_2023',
    'Government pensions and allowances: Commonwealth rent assistance (no.) (Data year: 2023)': 'gov_rent_assist_count_2023',
    'Personal income: Total income earners (excl. Government pensions and allowances)(no.) (Data year: 2020)': 'personal_income_count_2020',
    'Personal income: Total income earners (excl. Government pensions and allowances): Median age (years) (Data year: 2020)': 'personal_income_median_age_2020',
    'Personal income: Total income (excl. Government pensions and allowances) ($m) (Data year: 2020)': 'personal_total_income_millions_2020',
    'Personal income: Median total income (excl. Government pensions and allowances) ($) (Data year: 2020)': 'median_personal_total_income_2020',
    'Personal income: Mean total income (excl. Government pensions and allowances) ($) (Data year: 2020)': 'mean_personal_total_income_2020',
    'Personal income: Total income (excl. Government pensions and allowances): Gini coefficient (Data year: 2020)' : 'gini_coef_2020'
}

filtered_income.rename(columns=new_income_names, inplace=True)

# Get rid of trailing whitespaces for matching
filtered_income['sa2_name'] = filtered_income['sa2_name'].str.rstrip()

### Feature Selection & Column Renaming for Population Dataset

In [3]:
# Removing irrelevant columns
columns_to_drop = ['GCCSA code', 'GCCSA name', 'SA4 code', 'SA4 name', 'SA3 code', 'SA3 name']
population = population.drop(columns_to_drop, axis=1)

# Rename columns
new_population_cols = {
    'SA2 code': 'sa2_code',
    'SA2 name': 'sa2_name',
    'ERP at 30 June 2022 no.': 'erp_june_2022_count',
    'ERP at 30 June 2023 no.': 'erp_june_2023_count',
    'ERP change 2022-23 no.': 'erp_change_count',
    'ERP change 2022-23 %': 'erp_change_percentage',
    'Components of population change 2022-23 Natural increase no.': 'natural_increase_count',
    'Components of population change 2022-23 Net internal migration no.': 'net_internal_migration_count',
    'Components of population change 2022-23 Net overseas migration no.': 'net_overseas_migration_count',
    'Area (km2)': 'area_km2',
    'Population density 2023 (persons/km2)': 'pop_density_persons_km2'
}

population.rename(columns=new_population_cols, inplace=True)

# Drop empty / invalid SA2 names or codes
population = population[~population['sa2_code'].isna()]
population = population[~population['sa2_name'].isna()]

# Remove SA2 code as we don't need it anymore
population = population.drop('sa2_code', axis=1)

# Get rid of trailing whitespaces for matching
population['sa2_name'] = population['sa2_name'].str.rstrip()

### Merge Population & Income Dataset

In [4]:
demographics = pd.merge(population, filtered_income, on='sa2_name')

# Ensuring consistent schema
integer_cols = ['erp_june_2022_count', 'erp_june_2023_count', 'erp_change_count', 'natural_increase_count', 'net_internal_migration_count', 'net_overseas_migration_count', 'gov_age_pension_count_2023', 'gov_rent_assist_count_2023', 'personal_income_count_2020', 'personal_income_median_age_2020']
demographics[integer_cols] = demographics[integer_cols].astype('Int64')

demographics['sa2_name'] = demographics['sa2_name'].astype('str')
demographics['sa2_name'] = demographics['sa2_name'].str.lower()

# Save to csv
demographics.to_csv('../data/raw/demographics.csv', index=False)

### Load in Suburbs Shapefile

In [5]:
# Load Victoria suburbs shapefile for filtering
vic_suburbs_gdf = gpd.read_file('../data/map/Vic_Localities/gda2020_vicgrid/esrishape/whole_of_dataset/victoria/VMADMIN/LOCALITY_POLYGON.shp')
vic_suburbs_gdf = vic_suburbs_gdf.to_crs(epsg=4326)
vic_suburbs_gdf['GAZLOC_lower'] = vic_suburbs_gdf['GAZLOC'].str.lower()
print(list(vic_suburbs_gdf['GAZLOC_lower'].unique()))

['mollongghip', 'north blackwood', 'basalt', 'llanelly', 'murrabit west', 'springfield', 'waitchie', 'straten', 'gowanford', 'kunat', 'murrabit', 'lake charm', 'myall', 'pomonal', 'kenley', 'wandown', 'towan', 'bonnie doon', 'terip terip', 'emerald', 'healesville', 'thalloo', 'princetown', 'murrawee', 'natya', 'musk vale', 'piangil', 'beverford', 'cocamba', 'brucknell', 'ada', 'west wodonga', 'lerderderg', 'castle donnington', 'korweinguboora', 'robinvale', 'rocklyn', 'stanley', 'buffalo river', 'barrakee', 'silvan', 'kooloonong', 'bullarto south', 'bannerton', 'french island', 'wodonga', 'northwood', 'sandringham', 'monbulk', 'broadlands', 'metung', 'narrung', 'nyrraby', 'kiewa', 'grenville', 'lake powell', 'boundary bend', 'laceby', 'dooen', 'pimpinio', 'haven', 'brighton', 'lower norton', 'horsham', 'bungalally', 'drung', 'mckenzie creek', 'glendonald', 'kyvalley', 'speed', 'ultima', 'tresco', 'wangandary', 'oxley flats', 'tyenna', 'echuca', 'elaine', 'denver', 'nyah', 'turriff', 't

### Fix SA2 Names

In [6]:
# Define directional modifiers and the word 'surrounds' to be removed
directional_modifiers = [' - east', ' - west', ' - north', ' - south', ' - central', ' surrounds', ' (north)', ' (south)', ' (east)', ' (west)']
pattern = '|'.join([re.escape(suffix) for suffix in directional_modifiers])
demographics['sa2_name'] = demographics['sa2_name'].str.replace(pattern, '', regex=True)

In [7]:
# Split sa2_name where multiple names are separated by hyphens
demographics['sa2_name'] = demographics['sa2_name'].str.split(' - ')

# Explode the lists into separate rows
demographics_exploded = demographics.explode('sa2_name')
demographics_exploded = demographics_exploded.reset_index(drop=True)

In [8]:
# Mapping for the SA2 names to the correct suburbs
sa2_name_mapping = {
    'ballarat' : 'ballarat central',
    'flemington racecourse' : 'flemington',
    'southbank wharf' : 'south wharf',
    'port melbourne industrial' : 'port melbourne',
    'reservoir east' : 'reservoir',
    'reservoir west' : 'reservoir',
    'research warrandyte' : 'warrandyte',
    'essendon airport' : 'essendon',
    'gladstone parkmeadows' : 'gladstone park',
    'craigieburn west' : 'craigieburn',
    'wandin' : 'wandin north',
    'pakenham east' : 'pakenham',
    'pakenham west' : 'pakenham',
    'narre warren west' : 'narre warren',
    'berwick east' : 'berwick',
    'berwick west' : 'berwick',
    'point cook east' : 'point cook',
    'point cook west' : 'point cook',
    'truganina east' : 'truganina',
    'truganina west' : 'truganina',
    'melbourne cbd' : 'melbourne'
}

In [9]:
# Remove the "(vic.)" from sa2_name values
demographics_exploded['sa2_name'] = demographics_exploded['sa2_name'].str.replace(r'\s*\(vic\.\)', '', regex=True)
demographics_exploded['sa2_name'] = demographics_exploded['sa2_name'].replace(sa2_name_mapping)

In [10]:
# Merge the demographics with the suburb shapefile
merged_gdf = pd.merge(demographics_exploded, vic_suburbs_gdf, left_on='sa2_name', right_on='GAZLOC_lower', how='left')

In [11]:
# Checking which sa2 names aren't in suburbs
merged_gdf[merged_gdf['GAZLOC_lower'].isna()]

,sa2_name,erp_june_2022_count,erp_june_2023_count,erp_change_count,erp_change_percentage,natural_increase_count,net_internal_migration_count,net_overseas_migration_count,area_km2,pop_density_persons_km2,...,LOCALITY,GAZLOC,VICNAMESID,TASK_ID,PFI_CR,UFI_OLD,UFI_CR,LABEL_USE_,geometry,GAZLOC_lower
22,golden plains,4894,4940,46,0.9,19,16,11,922.1,5.4,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,None,NaN
46,loddon,7274,7256,-18,-0.2,-33,-13,28,6193.3,1.2,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,None,NaN
48,golden plains,8246,8467,221,2.7,66,132,23,1484.5,5.7,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,None,NaN
87,upper yarra valley,244,247,3,1.2,3,0,0,856.3,0.3,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,None,NaN
107,mount baw baw region,6634,6689,55,0.8,6,26,23,2753.3,2.4,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,None,NaN
110,alps,0,0,0,0.0,0,0,0,2071.3,0.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,None,NaN
114,lake king,0,0,0,0.0,0,0,0,93.9,0.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,None,NaN
122,phillip island,14074,14273,199,1.4,-55,176,78,100.6,141.9,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,None,NaN
134,alps,23,23,0,0.0,0,0,0,2955.7,0.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,None,NaN
167,industrial,0,0,0,0.0,0,0,0,6.2,0.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,None,NaN


### Impute Null Values Based on Nearest Suburbs

In [12]:
# Make GeoDataFrame to check nearest suburbs
merged_gdf = gpd.GeoDataFrame(merged_gdf, geometry='geometry')

# Function to calculate the nearest neighbors and impute the NaN values
def impute_missing_values(row, merged_gdf, columns_to_impute):
    # Find the geometry of the current row
    geometry = row['geometry']
    # Ensure geometry is valid and calculate distances only for non-null geometries
    if geometry and not pd.isnull(geometry):
        # Get nearby suburbs 
        nearby_suburbs = merged_gdf[merged_gdf.geometry.distance(geometry) <= 10000]  # 10 km buffer
        # For each column with missing values, calculate the mean from the nearby suburbs and impute
        for col in columns_to_impute:
            if pd.isna(row[col]) and not nearby_suburbs.empty:
                row[col] = nearby_suburbs[col].mean()
    
    return row

# Ensure both GeoDataFrame objects have valid geometries and CRS
merged_gdf = merged_gdf.set_crs(epsg=4326)  
columns_to_impute = [
    'gov_age_pension_count_2023', 'gov_rent_assist_count_2023', 'personal_income_count_2020', 
    'personal_income_median_age_2020', 'personal_total_income_millions_2020', 
    'median_personal_total_income_2020', 'mean_personal_total_income_2020', 'gini_coef_2020'
]
demographics_imputed = merged_gdf.apply(impute_missing_values, axis=1, merged_gdf=merged_gdf, columns_to_impute=columns_to_impute)

In [13]:
# Define the columns for which values should be copied over
columns_to_copy = [
    'gov_age_pension_count_2023', 'gov_rent_assist_count_2023',
    'personal_income_count_2020', 'personal_income_median_age_2020', 
    'personal_total_income_millions_2020', 'median_personal_total_income_2020', 
    'mean_personal_total_income_2020', 'gini_coef_2020'
]

# Function to find similar names and copy values over
def copy_values_from_similar_names(row, df, columns_to_copy):
    # Convert the current sa2_name to lowercase for easier matching
    sa2_name_lower = row['sa2_name'].lower()
    
    # Find rows where the sa2_name contains the base name (ignoring "east", "west", etc.)
    base_name = sa2_name_lower.split()[0]  # Take the first word as the base, e.g., 'ballarat'
    
    # Find potential matching rows in the dataframe
    similar_rows = df[df['sa2_name'].str.contains(base_name, case=False, na=False)]
    
    # If there are similar rows and the row itself has NaN values in the target columns, copy the values
    for col in columns_to_copy:
        if pd.isna(row[col]) and not similar_rows.empty:
            row[col] = similar_rows[col].mean()  # Use mean of similar rows for imputation
    
    return row

# Filter the rows where any of the target columns have NaN values
rows_with_nan = demographics_imputed[demographics_imputed[columns_to_copy].isna().any(axis=1)]

# Apply the imputation function only to rows with NaN values
demographics_imputed.loc[rows_with_nan.index] = rows_with_nan.apply(
    copy_values_from_similar_names, axis=1, df=demographics_imputed, columns_to_copy=columns_to_copy
)


### Take Mean and Median for Suburbs Unfound

In [14]:
# Define which columns to fill with mean and which with median
mean_columns = [
    'personal_total_income_millions_2020', 
    'mean_personal_total_income_2020', 'gini_coef_2020','gov_age_pension_count_2023',
    'gov_rent_assist_count_2023','personal_income_count_2020'
]

median_columns = [
    'median_personal_total_income_2020', 'personal_income_median_age_2020',
    'pop_density_persons_km2'
]

# Fill NaN values with the column mean
demographics_imputed[mean_columns] = demographics_imputed[mean_columns].fillna(demographics_imputed[mean_columns].mean())

# Fill NaN values with the column median
demographics_imputed[median_columns] = demographics_imputed[median_columns].fillna(demographics_imputed[median_columns].median())


In [15]:
# Define the columns to remove
columns_to_remove = [
    'UFI', 'PFI', 'LOCALITY', 'GAZLOC', 'VICNAMESID', 'TASK_ID', 'PFI_CR', 
    'UFI_OLD', 'UFI_CR', 'LABEL_USE_', 'geometry', 'GAZLOC_lower'
]

# Drop these columns from demographics_imputed
demographics_imputed = demographics_imputed.drop(columns=columns_to_remove)


### Combine duplicated SA2 names

In [16]:
# Columns to aggregate, and the type of aggregation (sum or mean)
aggregation_functions = {
    'erp_june_2022_count': 'sum',
    'erp_june_2023_count': 'sum',
    'erp_change_count': 'sum',
    'erp_change_percentage': 'mean',  # Mean for percentage
    'natural_increase_count': 'sum',
    'net_internal_migration_count': 'sum',
    'net_overseas_migration_count': 'sum',
    'area_km2': 'sum',
    'pop_density_persons_km2': 'mean',  # Population density mean
    'gov_age_pension_count_2023': 'sum',
    'gov_rent_assist_count_2023': 'sum',
    'personal_income_count_2020': 'sum',
    'personal_income_median_age_2020': 'mean',  # Median age mean
    'personal_total_income_millions_2020': 'sum',
    'median_personal_total_income_2020': 'mean',
    'mean_personal_total_income_2020': 'mean',
    'gini_coef_2020': 'mean'
}

# Perform aggregation
aggregated_data = demographics_imputed.groupby('sa2_name').agg(aggregation_functions).reset_index()


In [19]:
# Save demographics in csv
aggregated_data.to_csv('../data/curated/demographics.csv', index=False)